# Agent-Based Model of Evans 1997 using Agents.jl (Gemini)
Tomás Aragón

This notebook replicates the Evans 1997 migraine cost-effectiveness analysis using an agent-based (microsimulation) approach with [Agents.jl](https://juliadynamics.github.io/Agents.jl/stable/). Each agent is a migraine patient who walks through the decision tree stochastically, accumulating costs and receiving a utility outcome.

## Conceptual comparison

| Approach | Method | Output |
|----------|--------|--------|
| **DecisionProgramming.jl** | Analytical: computes exact expected values via influence diagram | Exact E[Cost], E[Utility] |
| **Agents.jl (this notebook)** | Simulation: N patients traverse the tree stochastically | Monte Carlo estimates of E[Cost], E[Utility] |

With enough agents, the ABM converges to the analytical solution. The ABM approach additionally gives us the full distribution of individual-level outcomes, which enables probabilistic sensitivity analysis and heterogeneity extensions.

Sources:
- Evans, K. W., et al. "Economic Evaluation of Oral Sumatriptan Compared with Oral Caffeine/Ergotamine for Migraine." *PharmacoEconomics* 12, no. 5 (1997): 565–77.
- Briggs, A. H., K. Claxton, and M. Sculpher. *Decision Modelling for Health Economic Evaluation.* Oxford Univ. Press, 2011.

## Setup and model parameters

In [1]:
using Agents
using Statistics
using DataFrames

In [3]:
# ==========================================
# 1. Define the Agent
# ==========================================
@agent struct MigrainePatient(NoSpaceAgent)
    treatment::Symbol 
    state::Symbol     
    cost::Float64
    utility::Float64
end

# ==========================================
# 2. Define Model Constants
# ==========================================
# Costs 
const c_sumatriptan = 16.10 #
const c_caffeine = 1.32 #
const c_ed = 63.16 #
const c_hospital = 1093.0 #

# Utilities 
const u_relief_norecurrence = 1.0 #
const u_relief_recurrence = 0.9 #
const u_norelief_endures = -0.30 #
const u_norelief_ed = 0.1 #

# ==========================================
# 3. Define the Agent Step Function
# ==========================================
function patient_step!(agent, model)
    # If the patient has reached a terminal leaf, do nothing
    if agent.state == :Done
        return
    end

    if agent.state == :Start
        if agent.treatment == :Sumatriptan
            agent.cost += c_sumatriptan
            # Use abmrng(model) instead of model.rng for Agents.jl v6+
            agent.state = rand(abmrng(model)) < 0.558 ? :Relief : :NoRelief #
        elseif agent.treatment == :Caffeine
            agent.cost += c_caffeine
            agent.state = rand(abmrng(model)) < 0.379 ? :Relief : :NoRelief #
        end

    elseif agent.state == :Relief
        if agent.treatment == :Sumatriptan
            if rand(abmrng(model)) < 0.594 #
                agent.utility = u_relief_norecurrence
            else
                agent.cost += c_sumatriptan 
                agent.utility = u_relief_recurrence
            end
        elseif agent.treatment == :Caffeine
            if rand(abmrng(model)) < 0.703 #
                agent.utility = u_relief_norecurrence
            else
                agent.cost += c_caffeine 
                agent.utility = u_relief_recurrence
            end
        end
        agent.state = :Done

    elseif agent.state == :NoRelief
        if rand(abmrng(model)) < 0.92 #
            agent.utility = u_norelief_endures
            agent.state = :Done
        else
            agent.cost += c_ed
            agent.state = :ED
        end

    elseif agent.state == :ED
        if rand(abmrng(model)) < 0.998 #
            agent.utility = u_norelief_ed
        else
            agent.cost += c_hospital
            agent.utility = u_norelief_endures
        end
        agent.state = :Done
    end
end

# ==========================================
# 4. Model Initialization & Execution
# ==========================================
function run_migraine_abm(; n_patients_per_arm = 50000)
    model = StandardABM(MigrainePatient; agent_step! = patient_step!)

    for _ in 1:n_patients_per_arm
        add_agent!(model, :Sumatriptan, :Start, 0.0, 0.0)
    end
    
    for _ in 1:n_patients_per_arm
        add_agent!(model, :Caffeine, :Start, 0.0, 0.0)
    end

    # Step the model 4 times to ensure all agents reach the end of the tree
    step!(model, 4)

    # Collect the data manually for maximum speed
    df = DataFrame(
        treatment = [a.treatment for a in allagents(model)],
        cost = [a.cost for a in allagents(model)],
        utility = [a.utility for a in allagents(model)]
    )

    # Calculate Expected Values
    results = combine(groupby(df, :treatment), 
        :cost => mean => :Expected_Cost,
        :utility => mean => :Expected_Utility
    )
    
    return results
end

# Run the simulation
results_df = run_migraine_abm()
println(results_df)

# Calculate ICER
suma = subset(results_df, :treatment => x -> x .== :Sumatriptan)[1, :]
caff = subset(results_df, :treatment => x -> x .== :Caffeine)[1, :]

delta_cost = suma.Expected_Cost - caff.Expected_Cost
delta_utility = suma.Expected_Utility - caff.Expected_Utility

# Annualize QALY
icer = (delta_cost / delta_utility) * 365
println("Estimated ICER: \$$(round(icer, digits=2)) Can/QALY")

2×3 DataFrame
 Row │ treatment    Expected_Cost  Expected_Utility 
     │ Symbol       Float64        Float64          
─────┼──────────────────────────────────────────────
   1 │ Caffeine           4.81975          0.199554
   2 │ Sumatriptan       22.2393           0.415896
Estimated ICER: $29389.36 Can/QALY
